In [9]:
import pandas as pd
import numpy as np
import warnings
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_forecasting import (
    TemporalFusionTransformer,
    TimeSeriesDataSet,
    GroupNormalizer,
    MAE,
    QuantileLoss
)

In [10]:
def fill_nearest_within_range(df, group_cols, target_col, window=10):
    df = df.copy()
    
    for group_values, group_df in df.groupby(group_cols):
        valid_idx = group_df[group_df[target_col].notna()].index.to_list()
        all_idx = group_df.index.to_list()

        index_pos_map = {idx: pos for pos, idx in enumerate(all_idx)}

        for idx in all_idx:
            if pd.isna(df.at[idx, target_col]):
                pos = index_pos_map[idx]
                start_pos = max(pos - window, 0)
                end_pos = min(pos + window, len(all_idx) - 1)
                candidates = [i for i in valid_idx if start_pos <= index_pos_map[i] <= end_pos]
                if not candidates:
                    continue

                candidate_dist = [(abs(index_pos_map[c] - pos), c) for c in candidates]
                min_dist, closest_idx = min(candidate_dist, key=lambda x: x[0])
                df.at[idx, target_col] = df.at[closest_idx, target_col]

                
    return df


In [11]:
warnings.filterwarnings("ignore")

# --------------------
# 데이터 로드 및 전처리
# --------------------
data = pd.read_csv('../data/test.csv', dtype={"evse_name": str, "station_location": str})

# UNIX timestamp → datetime (초 단위)
data["last_charge_end_time_ts"] = pd.to_datetime(data["last_charge_end_time_ts"], unit='s', errors='coerce').sort_values()

data = data.dropna(subset=["last_charge_end_time_ts", "evse_name", "station_location"])
data = data[data["last_charge_end_time_ts"] >= pd.Timestamp("2000-01-01")]




# 조합별 모든 time_idx 채우기 (Missing timestep 처리)
group_cols = ["station_location", "evse_name"]
unique_groups = data[group_cols].drop_duplicates()
time_index = pd.DataFrame()
#첫날 부터 30분씩 타임 슬롯 만들기
for idx, row in unique_groups.iterrows():
    loc = row["station_location"]
    evse = row["evse_name"]
    # 그룹별 유효 시간만 포함하여 DataFrame 생성

    # 해당 그룹의 데이터 필터링
    group_data = data[(data["station_location"] == loc) & (data["evse_name"] == evse)]

    # 그룹의 시간 범위 (min, max)
    start_time = group_data["last_charge_end_time_ts"].min().floor('30T')
    end_time = group_data["last_charge_end_time_ts"].max().ceil('30T')
    # 30분 간격 시간 생성
    time_index['store_timestamp'] = pd.date_range(start=start_time, end=end_time, freq='30T')
    time_index['evse_type'] = group_data['evse_type'][0]
    time_index['supports_discharge'] = group_data['supports_discharge'][0]
    time_index['weekday'] = time_index['store_timestamp'].dt.weekday
    time_index['month'] = time_index['store_timestamp'].dt.month
    
    print(time_index)


          store_timestamp evse_type supports_discharge  weekday  month
0     2024-07-01 00:00:00        FC                  n        0      7
1     2024-07-01 00:30:00        FC                  n        0      7
2     2024-07-01 01:00:00        FC                  n        0      7
3     2024-07-01 01:30:00        FC                  n        0      7
4     2024-07-01 02:00:00        FC                  n        0      7
...                   ...       ...                ...      ...    ...
18276 2025-07-16 18:00:00        FC                  n        2      7
18277 2025-07-16 18:30:00        FC                  n        2      7
18278 2025-07-16 19:00:00        FC                  n        2      7
18279 2025-07-16 19:30:00        FC                  n        2      7
18280 2025-07-16 20:00:00        FC                  n        2      7

[18281 rows x 5 columns]


In [12]:
# 두 시간 컬럼 모두 datetime 타입 확인 및 정렬
df_store = time_index.sort_values('store_timestamp').reset_index(drop=True)
df_charge = data.sort_values('last_charge_end_time_ts').reset_index(drop=True)

# merge_asof: 가장 가까운 ‘지나간’ 시간 기준으로 병합, 방향 지정 가능
merged_df = pd.merge_asof(
    df_store,
    df_charge,
    left_on='store_timestamp',
    right_on='last_charge_end_time_ts',
    direction='backward',   # 가장 가까운 시간으로 병합
    tolerance=pd.Timedelta('30min')  # 허용 오차 시간 범위 (예: 30분)
)
merged_df['last_charge_end_time_ts'] = merged_df['last_charge_end_time_ts'].fillna(method='ffill')
print(merged_df)

          store_timestamp evse_type_x supports_discharge_x  weekday_x  month  \
0     2024-07-01 00:00:00          FC                    n          0      7   
1     2024-07-01 00:30:00          FC                    n          0      7   
2     2024-07-01 01:00:00          FC                    n          0      7   
3     2024-07-01 01:30:00          FC                    n          0      7   
4     2024-07-01 02:00:00          FC                    n          0      7   
...                   ...         ...                  ...        ...    ...   
18276 2025-07-16 18:00:00          FC                    n          2      7   
18277 2025-07-16 18:30:00          FC                    n          2      7   
18278 2025-07-16 19:00:00          FC                    n          2      7   
18279 2025-07-16 19:30:00          FC                    n          2      7   
18280 2025-07-16 20:00:00          FC                    n          2      7   

      last_charge_end_time_ts  connecti

In [13]:
# timestamp 컬럼은 datetime, 시간 컬럼은 unix timestamp float 형식이라고 가정
# unix timestamp를 datetime으로 변환 (편리한 비교를 위해)
for col in ['connection_start_time_ts', 'connection_end_time_ts', 'charging_start_time_ts', 'charging_end_time_ts']:
    merged_df[col + '_dt'] = pd.to_datetime(merged_df[col], unit='s', errors='coerce')

# 충전 구간을 의미하는 시간 범위를 가진 행만 선택
valid_rows = merged_df.dropna(subset=['connection_start_time_ts_dt', 'connection_end_time_ts_dt'])

# 중간에 NaN 값이 있는 행에 대해, 누락된 시간 데이터를 채우기 위한 함수 정의
def fill_time_rows(row, intervals_df):
    # store_timestamp가 포함된 충전 구간 찾기
    row_dt_st =pd.to_datetime(row['store_timestamp'], unit='s')
    matches = intervals_df[
        (intervals_df['connection_start_time_ts_dt'] <= row_dt_st) &
        (intervals_df['connection_end_time_ts_dt'] >= row_dt_st)
    ]
    if not matches.empty:
        # 구간에 해당하는 첫번째 행 정보로 시간 컬럼 채우기
        matched = matches.iloc[0]
        for col in ['connection_start_time_ts', 'connection_end_time_ts'
                    ,'charging_start_time_ts', 'charging_end_time_ts'
                    ,'expected_departure_time_ts','expected_departure_time_missing'
                    ,'idle_time_ts','expected_usage_duration_ts','expected_usage_duration_ts'
                    ,'expected_usage_duration_ts','expected_time_diff_missing','actual_usage_duration_ts'
                    ,'actual_charging_duration_ts','actual_charging_duration_missing','start_delay_duration_ts'
                    ,'start_delay_duration_missing','post_charge_departure_delay_ts','post_charge_departure_delay_missing','usage_departure_time_diff_ts'
                    ,'usage_departure_time_diff_missing','delivered_kwh','requested_kwh','kwh_request_diff','kwh_per_usage_time']:
            if pd.isna(row[col]):
                row[col] = matched[col]

        for col_dt in ['connection_start_time_ts_dt', 'connection_end_time_ts_dt', 'charging_start_time_ts_dt', 'charging_end_time_ts_dt']:
            if pd.isna(row[col_dt]):
                row[col_dt] = matched[col_dt]
        for col_bool in ['charging_start_time_missing', 'charging_end_time_missing'
                       , 'duration_per_kwh_missing', 'kwh_per_usage_time_missing']:
            if pd.isna(row[col_bool]):
                row[col_bool] = matched[col_bool]
        for col_name in ['station_location', 'evse_name','evse_type_y','supports_discharge_y','scheduled_charge']:
            if pd.isna(row[col_name]):
                row[col_name] = matched[col_name]
        for col_cat in ['weekday_y', 'usage_departure_range','post_charge_departure_range','cluster']:
            if pd.isna(row[col_cat]):
                row[col_cat] = matched[col_cat]     

    return row

# apply 함수로 NaN 행들에 대해 채우기 수행
data_filled = merged_df.apply(lambda r: fill_time_rows(r, valid_rows), axis=1)

# 만약 unix timestamp로 원복이 필요하면 아래처럼 변환 가능
def datetime_to_unix(dt):
    if pd.isna(dt):
        return np.nan  # 결측 유지
    else:
        return int(dt.value // 10**9)  # datetime64[ns] -> 초 단위 unix timestamp

# 결과 확인
data_filled

,store_timestamp,evse_type_x,supports_discharge_x,weekday_x,month,last_charge_end_time_ts,connection_start_time_ts,charging_start_time_ts,charging_start_time_missing,charging_end_time_ts,...,supports_discharge_y,scheduled_charge,weekday_y,usage_departure_range,post_charge_departure_range,cluster,connection_start_time_ts_dt,connection_end_time_ts_dt,charging_start_time_ts_dt,charging_end_time_ts_dt
0,2024-07-01 00:00:00,FC,n,0,7,2024-07-01 00:00:00,1.719924e+09,1.719925e+09,False,1.719925e+09,...,n,n,1.0,0.0,0.0,0.0,2024-07-02 12:37:00,2024-07-02 13:24:52,2024-07-02 12:54:00,2024-07-02 13:04:00
1,2024-07-01 00:30:00,FC,n,0,7,2024-07-01 00:00:00,1.719924e+09,1.719925e+09,False,1.719925e+09,...,n,n,1.0,0.0,0.0,0.0,2024-07-02 12:37:00,2024-07-02 13:24:52,2024-07-02 12:54:00,2024-07-02 13:04:00
2,2024-07-01 01:00:00,FC,n,0,7,2024-07-01 00:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
3,2024-07-01 01:30:00,FC,n,0,7,2024-07-01 00:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
4,2024-07-01 02:00:00,FC,n,0,7,2024-07-01 00:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18276,2025-07-16 18:00:00,FC,n,2,7,2025-07-15 05:04:24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
18277,2025-07-16 18:30:00,FC,n,2,7,2025-07-15 05:04:24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
18278,2025-07-16 19:00:00,FC,n,2,7,2025-07-15 05:04:24,1.752692e+09,1.752693e+09,False,1.752694e+09,...,n,n,2.0,0.0,0.0,0.0,2025-07-16 18:59:24,2025-07-16 19:40:49,2025-07-16 19:07:24,2025-07-16 19:21:24
18279,2025-07-16 19:30:00,FC,n,2,7,2025-07-15 05:04:24,1.752692e+09,1.752693e+09,False,1.752694e+09,...,n,n,2.0,0.0,0.0,0.0,2025-07-16 18:59:24,2025-07-16 19:40:49,2025-07-16 19:07:24,2025-07-16 19:21:24


In [15]:
data_filled.tocsv('data_full.csv')

AttributeError: 'DataFrame' object has no attribute 'tocsv'